# DCOPF Verification

Prepared by [Jinning Wang](https://jinningwang.github.io).

## Conclusion

For test cases, DCOPF results from AMS are identical to that from MATPOWER.

In [1]:
import datetime

import numpy as np

import ams
import andes

import pypower.api as pyp

In [2]:
print("Last run time:", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

print(f'ams: {ams.__version__}')

Last run time: 2024-03-25 21:12:07
ams: 0.9.4.post35.dev0+g2a7bc56


In [3]:
ams.config_logger(stream_level=30)

## Generator Output

Using built-in MATPOWER cases as inputs.

In [4]:
cases = [
    ams.get_case('matpower/case14.m'),
    ams.get_case('matpower/case39.m'),
    ams.get_case('matpower/case118.m'),
    ams.get_case('npcc/npcc.m'),
    ams.get_case('wecc/wecc.m'),
    ams.get_case('matpower/case300.m'),]

case_names = [case.split('/')[-1].split('.')[0] for case in cases]

In [5]:
ams_obj = np.zeros(len(cases))
pyp_obj = np.zeros(len(cases))

for i, case in enumerate(cases):
    sp = ams.load(case, setup=True)
    sp.DCOPF.init()
    sp.DCOPF.run(solver='ECOS')
    ams_obj[i] = sp.DCOPF.obj.v

    ppc = ams.io.pypower.system2ppc(sp)
    ppopt = pyp.ppoption(VERBOSE=0, OUT_ALL=0, PF_ALG=1, OPF_ALG_DC=200)
    ppc_sol = pyp.rundcopf(ppc, ppopt)
    pyp_obj[i] = ppc_sol['f']

<DCOPF> solved as optimal in 0.0100 seconds, converged in 13 iterations with ECOS.
<DCOPF> solved as optimal in 0.0143 seconds, converged in 17 iterations with ECOS.
<DCOPF> solved as optimal in 0.0219 seconds, converged in 15 iterations with ECOS.
<DCOPF> solved as optimal in 0.0236 seconds, converged in 17 iterations with ECOS.
<DCOPF> solved as optimal in 0.0198 seconds, converged in 17 iterations with ECOS.
<DCOPF> solved as optimal in 0.0348 seconds, converged in 17 iterations with ECOS.


In [6]:
np.allclose(ams_obj, pyp_obj, atol=1e-6)

True

## LMP

In [7]:
sp2 = ams.load(ams.get_case('pglib/pglib_opf_case39_epri__api.m'),
               setup=True,
               no_output=True,
               default_config=True)

In [8]:
sp2.DCOPF.run(solver='ECOS')

<DCOPF> solved as optimal in 0.0125 seconds, converged in 14 iterations with ECOS.


True

In [9]:
ppc2 = ams.io.pypower.system2ppc(sp2)
ppc2_sol = pyp.rundcopf(ppc2, ppopt)

Nodal price

In [10]:
np.allclose(sp2.DCOPF.pi.v / sp2.config.mva,
            ppc2_sol['bus'][:, 13],
            atol=1e-6)

True

Bus angle

In [11]:
np.allclose(sp2.DCOPF.aBus.v * andes.shared.rad2deg,
            ppc2_sol['bus'][:, 8],
            atol=1e-6)

True